In [1]:
library(broom)
library(dplyr)

In [2]:
set.seed(42)
n <- 20000

In [3]:
eps1 <- rnorm(n)
eps2 <- rnorm(n)
eps3 <- rnorm(n)
epsx <- rnorm(n)
epsy <- rnorm(n)

In [4]:
Z1 <- eps1
Z3 <- eps3
Z2 <- Z3 + eps2
X <- Z1 + Z2 + epsx
Y <- X + Z1 + Z2 + Z3 + epsy

In [5]:
data <- data.frame(Y, X, Z1, Z2, Z3)

In [6]:
run_ols <- function(vars) {
  formula <- as.formula(paste("Y ~", paste(vars, collapse = " + ")))
  model <- lm(formula, data = data)
  coefs <- summary(model)$coefficients
  beta <- coefs["X", "Estimate"]
  se <- coefs["X", "Std. Error"]
  z <- 2.5758  # IC 99%
  ci_low <- beta - z * se
  ci_high <- beta + z * se
  return(list(model = model, beta = beta, se = se, ci_low = ci_low, ci_high = ci_high))
}

In [7]:
modelos <- list(
  c("X"),
  c("X", "Z1"),
  c("X", "Z2"),
  c("X", "Z1", "Z2"),
  c("X", "Z1", "Z2", "Z3")
)

In [8]:
resultados <- lapply(modelos, run_ols)

In [9]:
tabla <- data.frame(
  Regresion = c("Y~X", "Y~X+Z1", "Y~X+Z2", "Y~X+Z1+Z2", "Y~X+Z1+Z2+Z3"),
  Beta_X = sapply(resultados, function(x) x$beta),
  SE = sapply(resultados, function(x) x$se),
  CI_low = sapply(resultados, function(x) x$ci_low),
  CI_high = sapply(resultados, function(x) x$ci_high)
)

print("RESULTADOS DE LAS REGRESIONES (IC 99%)")
print(tabla)

In [10]:
png("efecto_X_sobre_Y.png", width = 900, height = 700)

with(tabla, {
  plot(Beta_X, ylim = c(min(CI_low), max(CI_high)),
       pch = 19, col = "orange", xaxt = "n",
       ylab = "Estimador de β_X", xlab = "",
       main = "Estimaciones del efecto de X sobre Y con IC 99%")
  arrows(1:length(Beta_X), CI_low, 1:length(Beta_X), CI_high,
         angle = 90, code = 3, length = 0.05, col = "orange")
  abline(h = 1, lty = 2)
  axis(1, at = 1:length(Beta_X), labels = tabla$Regresion)
})

dev.off()

El gráfico muestra las estimaciones del efecto de X sobre Y obtenidas bajo distintas especificaciones de control, junto con sus intervalos de confianza al 99%. Se observa que las regresiones Y~X, Y~X+Z1 y Y~X+Z2 producen estimaciones claramente superiores al valor verdadero (β = 1), lo cual indica sesgo por variables omitidas. Esto ocurre porque cada una de estas especificaciones deja sin controlar al menos una variable confusora Z1 o Z2, manteniendo abiertos los caminos de confusión entre X y Y.

En cambio, las regresiones Y~X+Z1+Z2 y Y~X+Z1+Z2+Z3 entregan estimaciones cercanas al valor real de 1, mostrando que controlar simultáneamente por Z1 y Z2 bloquea todos los backdoors y permite identificar correctamente el efecto causal de X sobre Y. No obstante, incluir Z3 no mejora la validez de la estimación, ya que esta variable es un ancestro de Z2 y no un confounder directo, lo que solo introduce colinealidad y reduce ligeramente la precisión del estimador.